<a href="https://colab.research.google.com/github/JK-the-Ko/AI-and-DL/blob/main/Week14/%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5%EA%B3%BC%EB%94%A5%EB%9F%AC%EB%8B%9D_PyTorch_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generative Adversarial Networks Using PyTorch Framework

## Check NVIDIA GPU Setting

In [ ]:
!nvidia-smi

## Load MNIST Dataset

In [ ]:
from torchvision.datasets import MNIST
from torchvision import transforms

In [ ]:
trainDataset = MNIST(root="content", 
                     train=True, 
                     transform=transforms.Compose([transforms.Resize((32,32)), transforms.ToTensor(), transforms.Normalize(0.5, 0.5)]),
                     download=True)
testDataset = MNIST(root="content", 
                    train=False, 
                    transform=transforms.Compose([transforms.Resize((32,32)), transforms.ToTensor(), transforms.Normalize(0.5, 0.5)]),
                    download=True)

## Vanilla GAN

In [ ]:
from torch import nn
import torch.nn.functional as F

### Generator

In [ ]:
class Generator(nn.Module) :
  def __init__(self, opt) :
    super(Generator, self).__init__()

    zDim = opt["zDim"]
    self.targetDim, self.targetSize = opt["targetDim"], opt["targetSize"]

    self.block0 = nn.Sequential(nn.Linear(zDim, 128),
                                nn.LeakyReLU(0.2))
    self.block1 = nn.Sequential(nn.Linear(128, 256),
                                nn.BatchNorm1d(256, 0.8),
                                nn.LeakyReLU(0.2))
    self.block2 = nn.Sequential(nn.Linear(256, 512),
                                nn.BatchNorm1d(512, 0.8),
                                nn.LeakyReLU(0.2))
    self.block3 = nn.Sequential(nn.Linear(512, 1024),
                                nn.BatchNorm1d(1024, 0.8),
                                nn.LeakyReLU(0.2))
    self.block4 = nn.Sequential(nn.Linear(1024, (self.targetSize**2)*self.targetDim),
                                nn.Tanh())

  def forward(self, z) :
    output = self.block0(z)
    output = self.block1(output)
    output = self.block2(output)
    output = self.block3(output)
    output = self.block4(output)
    output = output.view(-1, self.targetDim, self.targetSize, self.targetSize)

    return output

### Discriminator

In [ ]:
class Discriminator(nn.Module) :
  def __init__(self, opt) :
    super(Discriminator, self).__init__()

    self.targetDim, self.targetSize = opt["targetDim"], opt["targetSize"]

    self.model = nn.Sequential(nn.Linear((self.targetSize**2)*self.targetDim, 512),
                               nn.LeakyReLU(0.2),
                               nn.Linear(512, 256),
                               nn.LeakyReLU(0.2),
                               nn.Linear(256, 1),
                               nn.Sigmoid())

  def forward(self, input) :
    input = input.view(-1, (self.targetSize**2)*self.targetDim)
    output = self.model(input)

    return output

## Train DL Model

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch import optim
from torch.autograd import Variable

from torchsummary import summary

from tqdm import tqdm

### Fix Seed

In [ ]:
import random
import numpy as np

In [ ]:
def fixSeed(seed) :
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

## Create Average Meter Instance

In [ ]:
class AverageMeter(object):
  def __init__(self):
    self.reset()

  def reset(self):
    self.val = 0
    self.avg = 0
    self.sum = 0
    self.count = 0

  def update(self, val, n=1):
    self.val = val
    self.sum += val*n
    self.count += n
    self.avg = self.sum / self.count

## Training Code as a Function (Abstraction)

In [ ]:
def train(opt, trainDataset, testDataset, modelG, modelD, criterion) :
  fixSeed(opt["seed"])

  trainDataLoader = DataLoader(trainDataset, batch_size=opt["batchSize"], shuffle=True, drop_last=True)
  testDataLoader = DataLoader(testDataset, batch_size=opt["batchSize"], shuffle=False, drop_last=False)

  fixSeed(opt["seed"])
  modelG = Generator(opt)
  modelD = Discriminator(opt)
  if opt["isCUDA"] :
    modelG = modelG.cuda()
    modelD = modelD.cuda()

  optimizerG = optim.Adam(modelG.parameters(), lr=opt["lrG"])
  optimizerD = optim.Adam(modelD.parameters(), lr=opt["lrD"])

  Tensor = torch.cuda.FloatTensor if opt["isCUDA"] else torch.FloatTensor
  
  trainGLoss, testGLoss = AverageMeter(), AverageMeter()
  trainGLossList, testGLossList = [], []
  bestLoss = torch.inf

  for epoch in range(1, opt["epochs"]+1) :
    trainBar = tqdm(trainDataLoader)
    trainGLoss.reset()

    for data in trainBar :
      input, target = data
      if opt["isCUDA"] :
        input = input.cuda()

      valid = Variable(Tensor(input.size(0), 1).fill_(1.0), requires_grad=False)
      fake = Variable(Tensor(input.size(0), 1).fill_(0.0), requires_grad=False)

      optimizerG.zero_grad()
      z = Variable(Tensor(np.random.normal(0, 1, (opt["batchSize"], opt["zDim"]))))
      lossG = criterion(modelD(modelG(z)), valid)
      lossG.backward()
      optimizerG.step()

      optimizerD.zero_grad()
      z = Variable(Tensor(np.random.normal(0, 1, (opt["batchSize"], opt["zDim"]))))
      lossDReal = criterion(modelD(input), valid)
      lossDFake = criterion(modelD(modelG(z).detach()), fake)
      lossD = (lossDReal+lossDFake)/2
      lossD.backward()
      optimizerD.step()

      trainGLoss.update(lossG.item(), opt["batchSize"])
      trainBar.set_description(desc=f"[{epoch}/{opt['epochs']}] [Train] < Loss(G):{trainGLoss.avg:.6f} >")

    trainGLossList.append(trainGLoss.avg)

    testBar = tqdm(testDataLoader)
    testGLoss.reset()

    for data in testBar :
      input, target = data
      if opt["isCUDA"] :
        input = input.cuda()

      modelG.eval(), modelD.eval()
      with torch.no_grad() :
        z = Variable(Tensor(np.random.normal(0, 1, (opt["batchSize"], opt["zDim"]))))
        lossG = criterion(modelD(modelG(z)), valid)

        testGLoss.update(lossG.item(), opt["batchSize"])

        testBar.set_description(desc=f"[{epoch}/{opt['epochs']}] [Test] < Loss:{testGLoss.avg:.6f} >")

    testGLossList.append(testGLoss.avg)

    if testGLoss.avg < bestLoss :
      bestLoss = testGLoss.avg
      torch.save(modelG.state_dict(), opt["saveRoot"]+"bestModel.pth")

    torch.save(modelG.state_dict(), opt["saveRoot"]+"latestModel.pth")

  return (trainGLossList, testGLossList)

## Create Training Option (Hyperparameter) Dictionary

In [ ]:
opt = {"saveRoot":"/content/",
       "targetSize":32,
       "seed":42,
       "targetDim":1,
       "zDim":100,
       "batchSize":64, 
       "lrG":1e-4,
       "lrD":4e-4, 
       "epochs":10, 
       "isCUDA":torch.cuda.is_available()}

## Train Model

In [ ]:
lossList = train(opt, trainDataset, testDataset, Generator, Discriminator, nn.BCELoss())

## Plot Training vs. Test Loss Graph

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(20,10))

plt.plot(np.arange(0, opt["epochs"], 1), lossList[0], label="Training Loss")
plt.plot(np.arange(0, opt["epochs"], 1), lossList[1], label="Test Loss")

plt.xlabel("Epoch")
plt.ylabel("BCE Loss")
plt.legend(loc="best")

plt.show()

## Generate Images

### Load Trained Model

In [ ]:
weights = torch.load("/content/latestModel.pth")

model = Generator(opt)
model.load_state_dict(weights)
if opt["isCUDA"] :
  model = model.cuda()

### Get Model Structure

In [ ]:
print(model)

### Inference

In [ ]:
numImage = 64

In [ ]:
Tensor = torch.cuda.FloatTensor if opt["isCUDA"] else torch.FloatTensor
z = Variable(Tensor(np.random.normal(0, 1, (numImage, opt["zDim"]))))

In [ ]:
gen = model(z)

### Visualize Result

In [ ]:
from torchvision.utils import make_grid

In [ ]:
plt.imshow(make_grid(gen, normalize=True).permute(1,2,0).detach().cpu())
plt.show()

### Load Trained Model

In [ ]:
weights = torch.load("/content/latestModel-200.pth")

model = Generator(opt)
model.load_state_dict(weights)
if opt["isCUDA"] :
  model = model.cuda()

### Inference

In [ ]:
numImage = 64

In [ ]:
Tensor = torch.cuda.FloatTensor if opt["isCUDA"] else torch.FloatTensor
z = Variable(Tensor(np.random.normal(0, 1, (numImage, opt["zDim"]))))

In [ ]:
gen = model(z)

### Visualize Result

In [ ]:
plt.imshow(make_grid(gen, normalize=True).permute(1,2,0).detach().cpu())
plt.show()